In [1]:
from agent.scrape import scrape_sklearn_website

In [2]:
# sklearn_data = scrape_sklearn_website()

In [3]:
import json

with open("data/sklearn_function_openai.json","r") as jsonfile:
    sklearn_data = json.load(jsonfile)

In [4]:
from agent.utils import build_no_summary_graph,get_parents_dict

In [5]:
sklearn_graph = build_no_summary_graph(sklearn_data)

In [6]:
parents_dict = get_parents_dict(sklearn_graph)

In [7]:
parents_dict['sklearn.model_selection']['sklearn.model_selection#Splitters']

['K-fold iterator variant with non-overlapping groups. Each group will appear exactly once in the test set across all folds (the\nnumber of distinct groups has to be at least equal to the number of folds). The folds are approximately balanced in the sense that the number of\nsamples is approximately the same in each test fold. Read more in the User Guide. For visualisation of cross-validation behaviour and\ncomparison between common scikit-learn split methods\nrefer to Visualizing cross-validation behavior in scikit-learn Number of folds. Must be at least 2. Changed in version 0.22: n_splits default value changed from 3 to 5. See also For splitting the data according to explicit domain-specific stratification of the dataset. Takes class information into account to avoid building folds with imbalanced class proportions (for binary or multiclass classification tasks). Notes Groups appear in an arbitrary order throughout the folds.  Get metadata routing of this object. Please check User G

## LCEL TO SUMMARIZE THE DATA

In [8]:
# ans = summary_chain.invoke({"descriptions":"\n\n".join(parents_dict['sklearn']['sklearn#defaults'])})

In [9]:
# ans = await summary_chain.abatch([{"descriptions":"\n\n".join(parents_dict['sklearn']['sklearn#defaults'])}],config={"max_concurrency": 5})
# ans

In [10]:
from typing import List
def split_description(parent_text:List[str],MAX_WORDS:int=500):
        split_s = []
        running_num_words = 0
        curr_func_string = ""
        for txt in parent_text:
            num_words = len(txt.split(" "))
            running_num_words += num_words
            if running_num_words > MAX_WORDS:
                running_num_words = num_words
                split_s.append(curr_func_string)
                curr_func_string = txt
            else:
                curr_func_string += txt + "\n"
        if split_s == []:
            split_s.append(curr_func_string)
        split_s = [s for s in split_s if s!=""]
        return split_s

In [11]:
parent_child_dict = {pn:{} for pn in parents_dict}

for parent_name,child_nodes in parents_dict.items():
    for child_name,child_texts in child_nodes.items():
        child_split_list = split_description(child_texts,500)
        parent_child_dict[parent_name].update({child_name:child_split_list})        

In [18]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/home/athekunal/Function Calling/openbb-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv,find_dotenv
# from langchain_openai_limiter import LimitAwaitChatOpenAI, ChooseKeyChatOpenAI

load_dotenv(find_dotenv(),override=True)
model = ChatOpenAI(model="gpt-3.5-turbo-0125",api_key=os.environ['OPENAI_API_KEY0'],max_retries=5)
# chat_model_limit_await = LimitAwaitChatOpenAI(
#     chat_openai=model,
#     limit_await_timeout=60.0,
#     limit_await_sleep=0.1,
# )
# chat_model_key_choose = ChooseKeyChatOpenAI(
#     chat_openai=chat_model_limit_await,
#     openai_api_keys=[
#         os.environ["OPENAI_API_KEY0"],
#         os.environ["OPENAI_API_KEY1"],
#     ]
# )

summary_prompt = ChatPromptTemplate.from_template(
"""
You are given a list of descriptions of different functions separated by new paragraph.
Your task is to summarize all the text into coherent and detailed summary that covers all the functions descriptions.
Be very diligent and make sure that no function description is left out of the final summary. 

Follow the following format

List of function descriptions: list of function descriptions to be summarized
Summary: summary of all the function descriptions

-----

List of functions descriptions: {descriptions}

Summary: 
"""
)

summary_chain = summary_prompt | model | StrOutputParser()
parent_summary_dict = {pn:{} for pn in parents_dict}

batch_list = []
for parent_name,child_node in parent_child_dict.items():
    for child_name,child_texts in child_node.items():
        for ctext in child_texts:
            batch_list.append({f"{parent_name}-->{child_name}":ctext})


description_list = [{"descriptions":list(desc.values())[0]} for desc in batch_list]
# summary_list = summary_chain.batch_as_completed(description_list,config={"max_concurrency":10})
# from tqdm import tqdm

# pbar = tqdm(total=len(batch_list))
# for sl in summary_list:
#     bl = batch_list[sl[0]]
#     parent_name = list(bl.keys())[0].split("-->")[0]
#     child_name = list(bl.keys())[0].split("-->")[1]
#     if child_name in parent_summary_dict[parent_name]:
#         parent_summary_dict[parent_name][child_name].append(sl[1])
#     else:
#         parent_summary_dict[parent_name].update({child_name:[sl[1]]})
#     pbar.update(1)

In [14]:
len(description_list),len(batch_list)

(353, 353)

{'Recently Deprecated-->Recently Deprecated#defaults': 'Get a list of all estimators from sklearn. This function crawls the module and gets all classes that inherit\nfrom BaseEstimator. Classes that are defined in test-modules are not\nincluded. Which kind of estimators should be returned. If None, no filter is\napplied and all estimators are returned.  Possible values are\n‘classifier’, ‘regressor’, ‘cluster’ and ‘transformer’ to get\nestimators only of these specific types, or a list of these to\nget the estimators that fit at least one of the types. List of (name, class), where name is the class name as string\nand class is the actual type of the class.\nGet a list of all functions from sklearn. List of (name, function), where name is the function name as\nstring and function is the actual function.\n'}

In [18]:
ans = summary_chain.batch_as_completed(description_list,config={"max_concurrency":10})

In [14]:
for i in ans:
    print(i)

(0, "The global scikit-learn configuration offers various options for customization. \n- The configuration includes settings for validation of finiteness, size of temporary arrays, printing of non-default parameters, display of estimators, row vectors per chunk, accelerated pairwise-distances reduction backend, Array API dispatching, output format of transform and fit_transform, metadata routing, validation of hyper-parameters' types and values, and retrieval of current configuration values.\n- Users can choose to skip validation for finiteness for faster processing, set a limit for temporary array size, control the printing of non-default parameters, display estimators as diagrams or text, adjust the number of row vectors per chunk, use the accelerated pairwise-distances reduction backend, enable Array API dispatching, configure output formats, enable metadata routing, and disable validation of hyper-parameters' types and values.\n- These configurations can be customized globally and 

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125",api_key=os.environ['OPENAI_API_KEY'],max_retries=5,model_kwargs={"batch_size":20})

In [ ]:
model.model_kwargs

{'batch_size': 20}